In [3]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from keras.utils.visualize_util import plot
from keras.optimizers import *
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
import csv
from sklearn.metrics import log_loss
from PIL import Image
import os
import random
from keras.applications.resnet50 import ResNet50
from keras.regularizers import l2, activity_l2
from keras.models import model_from_json
from keras.layers import Merge

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [2]:
def extract_zip():
    import zipfile
    zip_ref = zipfile.ZipFile('sample_submission_stg1.csv.zip', 'r')
    zip_ref.extractall('./')
    zip_ref.close()

In [3]:
train_path = './train/train/'
test_path = './test/test/'
cropped_images_test = './cropped_test/_gen/'
cropped_images_train = './cropped_train/_gen/'

In [4]:
data_gen_args = dict(rotation_range=90.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)
train_datagen = ImageDataGenerator(**data_gen_args)
test_datagen = ImageDataGenerator(**data_gen_args)
real_test_datagen = ImageDataGenerator(**data_gen_args)
train_generator = train_datagen.flow_from_directory(cropped_images_train, batch_size=16,color_mode='rgb')
validation_generator = test_datagen.flow_from_directory(cropped_images_test, batch_size=16,color_mode='rgb')
real_test = test_datagen.flow_from_directory(train_path, batch_size=16,color_mode='rgb')

Found 15929 images belonging to 8 classes.
Found 4025 images belonging to 8 classes.
Found 3777 images belonging to 8 classes.


In [5]:
def return_model():
    model = Sequential()
    model.add(Activation(activation="relu", input_shape=(3,None,None)))

    model.add(Convolution2D(64, 5, 5, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Convolution2D(64, 5, 5, border_mode='same', activation='relu'))

    model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))

    model.add(GlobalMaxPooling2D())
    model.add(Dense(512, activation='relu'))

    model.add(Dense(128, activation='relu'))

    model.add(Dense(6))
    model.add(Activation('softmax'))
    return model



In [4]:
def return_resnet50():
    base_model = ResNet50(weights='imagenet', include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(2048,init='normal', activation='relu')(x)
    x = Dropout(0.005)(x)
    x = Dense(512,init='normal', activation='relu')(x)
    x = Dropout(0.005)(x)
    x = Dense(512,init='normal', activation='relu')(x)
    x = Dropout(0.005)(x)
    x = Dense(128,init='normal', activation='relu')(x)
    x = Dropout(0.005)(x)
    predictions = Dense(8,init='normal', activation='softmax')(x)

    model = Model(input=base_model.input, output=predictions)

    L = len(base_model.layers)
    for layer in base_model.layers:
        layer.trainable = False

    return model
 


In [5]:
model = return_resnet50()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, None, None) 0                                            
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, None, None) 0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 64, None, None 9472        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 64, None, None 256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
def return_resnet50_t():

    base_model = ResNet50(weights='imagenet', include_top=False)
    x = base_model.output
    head = Model(input=base_model.input, output=GlobalAveragePooling2D()(x))
    
    left_branch = Sequential()
    left_branch.add(head)
    left_branch.add(Dense(32))

    right_branch = Sequential()
    right_branch.add(head)
    right_branch.add(Dense(32))

    merged = Merge([left_branch, right_branch], mode='concat')

    final_model = Sequential()
    final_model.add(merged)
    final_model.add(Dense(8, activation='softmax'))
    model = Model(input=base_model.input, output=final_model.output)

    L = len(base_model.layers)
    for layer in base_model.layers:
        layer.trainable = False

    return model

In [ ]:
model = return_resnet50()
opt = SGD(lr=1e-2, decay=1e-6, momentum=0.1, nesterov=True)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
nb_epoch = 500
# train the model on the new data for a few epochs
model.fit_generator(train_generator,
        samples_per_epoch=1024,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        verbose=0,
        nb_val_samples=256, callbacks=[remote])

In [ ]:
model.evaluate_generator(validation_generator,2000)

In [ ]:
model.save('model_8c_500e_R50_1')

In [ ]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model_8c_500e_R50_1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_8c_500e_R50_1.h5")
print("Saved model to disk")


In [15]:
# load json and create model
json_file = open('model_8c_500e_R50_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_8c_500e_R50_1.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
loaded_model.evaluate_generator(validation_generator,1000)

Loaded model from disk


[0.3044509439034358, 0.90674603174603174]

In [16]:
loaded_model.evaluate_generator(validation_generator,2000)

[0.309276511132413, 0.91488302638128427]

In [ ]:
print(basemodel)

In [ ]:
print(len(predictions[0]))
print(predictions[0])

In [ ]:
folder_test = './test/test/'

data_gen_args = dict(
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.,
    zoom_range=0.1,
    )

test_datagen = ImageDataGenerator(**data_gen_args)

test_generator = validation_datagen.flow_from_directory(folder_test, batch_size=16)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
input_path = './cropped_test'
def get_random_images(N):
    import collections
    obj = collections.defaultdict(list)
    d = collections.defaultdict(list)
    for files in os.listdir(input_path):
        for img in os.listdir(input_path+'/'+files):
            d[files].append(img)
    for key in d.keys():
        for i in range(N):
            obj[key].append(random.choice(d[key]))
    return obj
d = get_random_images(50) 
predictions = []
test_id = []
for key in d.keys():
    for img_name in d[key]:
        image_path = input_path+'/'+key+'/'+img_name
        img = img_to_array(Image.open(image_path))
        img = img.reshape((1,) + img.shape)
        results = model.predict(img)
        obj = np.array(results[0])
        obj = np.hstack((np.array(img_name[2:]).T,obj))
        predictions.append(obj)
        test_id.append(img_name[2:])

In [ ]:
import datetime
import pandas as pd
def create_submission(predictions, test_id, info):
    #result1 = pd.DataFrame(predictions, columns=['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT','image'])
    result1 = pd.DataFrame(predictions, columns=['image','ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])
    #result1.loc['image',:] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    result1.to_csv(sub_file, index=False)
#create_submission(predictions, test_id, '')